In [128]:
import pandas as pd
import os

In [129]:
# 1.1 Merge the ACS 5-Year data with the SF1 data
ACSDT5Y_2010_2022 = pd.read_csv(
    "/Users/koacow/BOSTON UNIVERSITY Dropbox/Ngoc Duy Khoa Cao/GLOB~S/Data/U.S. County Data/county_controls/data/ACSDT5Y_2010_2022/acs_2010_2022.csv",
    dtype={"FIPS5": str},
)
SF1_1980_2020 = pd.read_csv(
    "/Users/koacow/BOSTON UNIVERSITY Dropbox/Ngoc Duy Khoa Cao/GLOB~S/Data/U.S. County Data/county_controls/data/NHGIS_RACE_BY_COUNTY_1980_2020/sf1_1980_2020.csv"
    , dtype={"FIPS5": str}
)
ACS_SF1 = pd.merge(
    ACSDT5Y_2010_2022,
    SF1_1980_2020,
    on=["FIPS5", "year"],
    how="outer",
    suffixes=("_acs", "_sf1"),
)
ACS_SF1 = ACS_SF1.sort_values(by=["FIPS5", "year"])
ACS_SF1

,FIPS5,year,white_pop_acs,black_pop_acs,asian_pop_acs,median_household_income,total_pop_acs,other_pop_acs,bachelors_pop_acs,total_pop_sf1,white_pop_sf1,black_pop_sf1,asian_pop_sf1,other_pop_sf1,bachelors_pop_sf1
0,01001,1980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32259.0,24814.0,7225.0,82.0,138.0,2117.0
1,01001,1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34222.0,27144.0,6845.0,120.0,113.0,3026.0
2,01001,2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43671.0,35221.0,7473.0,200.0,777.0,4972.0
3,01001,2010,NaN,NaN,NaN,53255.0,NaN,NaN,NaN,54571.0,42855.0,9643.0,474.0,1599.0,NaN
4,01001,2011,NaN,NaN,NaN,53899.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51282,72153,2018,28189.0,1300.0,5.0,14954.0,36439.0,6945.0,6386.0,NaN,NaN,NaN,NaN,NaN,NaN
51283,72153,2019,26607.0,1346.0,6.0,14743.0,35428.0,7469.0,6304.0,NaN,NaN,NaN,NaN,NaN,NaN
51284,72153,2020,24931.0,1244.0,5.0,14813,34501.0,8321.0,6156.0,34172.0,7312.0,1617.0,25.0,25218.0,NaN
51285,72153,2021,24533.0,1282.0,0.0,16444,34704.0,8889.0,6651.0,NaN,NaN,NaN,NaN,NaN,NaN


In [130]:
# 1.1 Use the SF1 data where available, otherwise use the ACS data

ACS_SF1["total_pop"] = ACS_SF1["total_pop_sf1"].combine_first(
    ACS_SF1["total_pop_acs"]
)
ACS_SF1["white_pop"] = ACS_SF1["white_pop_sf1"].combine_first(
    ACS_SF1["white_pop_acs"]
)
ACS_SF1["black_pop"] = ACS_SF1["black_pop_sf1"].combine_first(
    ACS_SF1["black_pop_acs"]
)
ACS_SF1["asian_pop"] = ACS_SF1["asian_pop_sf1"].combine_first(
    ACS_SF1["asian_pop_acs"]
)
ACS_SF1["other_pop"] = ACS_SF1["other_pop_sf1"].combine_first(
    ACS_SF1["other_pop_acs"]
)
ACS_SF1["bachelors_pop"] = ACS_SF1["bachelors_pop_sf1"].combine_first(
    ACS_SF1["bachelors_pop_acs"]
)
ACS_SF1 = ACS_SF1.drop(
    columns=[
        "total_pop_sf1",
        "white_pop_sf1",
        "black_pop_sf1",
        "asian_pop_sf1",
        "other_pop_sf1",
        "bachelors_pop_sf1",
        "total_pop_acs",
        "white_pop_acs",
        "black_pop_acs",
        "asian_pop_acs",
        "other_pop_acs",
        "bachelors_pop_acs",
    ]
)
ACS_SF1

,FIPS5,year,median_household_income,total_pop,white_pop,black_pop,asian_pop,other_pop,bachelors_pop
0,01001,1980,NaN,32259.0,24814.0,7225.0,82.0,138.0,2117.0
1,01001,1990,NaN,34222.0,27144.0,6845.0,120.0,113.0,3026.0
2,01001,2000,NaN,43671.0,35221.0,7473.0,200.0,777.0,4972.0
3,01001,2010,53255.0,54571.0,42855.0,9643.0,474.0,1599.0,NaN
4,01001,2011,53899.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
51282,72153,2018,14954.0,36439.0,28189.0,1300.0,5.0,6945.0,6386.0
51283,72153,2019,14743.0,35428.0,26607.0,1346.0,6.0,7469.0,6304.0
51284,72153,2020,14813,34172.0,7312.0,1617.0,25.0,25218.0,6156.0
51285,72153,2021,16444,34704.0,24533.0,1282.0,0.0,8889.0,6651.0


In [131]:
# 1.2 Compute the percentage of each racial group
ACS_SF1["white_pct"] = ACS_SF1["white_pop"] / ACS_SF1["total_pop"]
ACS_SF1["black_pct"] = ACS_SF1["black_pop"] / ACS_SF1["total_pop"]
ACS_SF1["asian_pct"] = ACS_SF1["asian_pop"] / ACS_SF1["total_pop"]
ACS_SF1["other_pct"] = ACS_SF1["other_pop"] / ACS_SF1["total_pop"]
ACS_SF1 = ACS_SF1.drop(
    columns=["white_pop", "black_pop", "asian_pop", "other_pop"]
)

# 1.3 Compute the percentage of bachelor's or higher degree holders
ACS_SF1["bachelors_pct"] = ACS_SF1["bachelors_pop"] / ACS_SF1["total_pop"]
ACS_SF1 = ACS_SF1.drop(columns=["bachelors_pop"])
ACS_SF1

,FIPS5,year,median_household_income,total_pop,white_pct,black_pct,asian_pct,other_pct,bachelors_pct
0,01001,1980,NaN,32259.0,0.769212,0.223969,0.002542,0.004278,0.065625
1,01001,1990,NaN,34222.0,0.793174,0.200018,0.003507,0.003302,0.088423
2,01001,2000,NaN,43671.0,0.806508,0.171120,0.004580,0.017792,0.113851
3,01001,2010,53255.0,54571.0,0.785307,0.176706,0.008686,0.029301,NaN
4,01001,2011,53899.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
51282,72153,2018,14954.0,36439.0,0.773594,0.035676,0.000137,0.190592,0.175252
51283,72153,2019,14743.0,35428.0,0.751016,0.037993,0.000169,0.210822,0.177938
51284,72153,2020,14813,34172.0,0.213976,0.047319,0.000732,0.737973,0.180147
51285,72153,2021,16444,34704.0,0.706921,0.036941,0.000000,0.256138,0.191649


In [132]:
ACS_SF1.isna().sum()

FIPS5                         0
year                          0
median_household_income    9421
total_pop                    11
white_pct                    11
black_pct                    11
asian_pct                    11
other_pct                    11
bachelors_pct              6463
dtype: int64

In [133]:
# 2.1 Create a master panel dataframe for all year x FIPS5 combinations
years = range(1980, 2023)
fips_codes = ACS_SF1["FIPS5"].unique()
master_panel = pd.DataFrame(
    [(fips, year) for fips in fips_codes for year in years],
    columns=["FIPS5", "year"],
)
master_panel["FIPS5"] = master_panel["FIPS5"].astype(str).str.zfill(5)
master_panel["year"] = master_panel["year"].astype(int)
master_panel


,FIPS5,year
0,01001,1980
1,01001,1981
2,01001,1982
3,01001,1983
4,01001,1984
...,...,...
139487,72153,2018
139488,72153,2019
139489,72153,2020
139490,72153,2021


In [134]:
# 2.2 Left join the master panel with the ACS_SF1 dataframe
master_panel = master_panel.merge(
    ACS_SF1,
    on=["FIPS5", "year"],
    how="left",
)
master_panel

,FIPS5,year,median_household_income,total_pop,white_pct,black_pct,asian_pct,other_pct,bachelors_pct
0,01001,1980,NaN,32259.0,0.769212,0.223969,0.002542,0.004278,0.065625
1,01001,1981,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01001,1982,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01001,1983,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01001,1984,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
139487,72153,2018,14954.0,36439.0,0.773594,0.035676,0.000137,0.190592,0.175252
139488,72153,2019,14743.0,35428.0,0.751016,0.037993,0.000169,0.210822,0.177938
139489,72153,2020,14813,34172.0,0.213976,0.047319,0.000732,0.737973,0.180147
139490,72153,2021,16444,34704.0,0.706921,0.036941,0.000000,0.256138,0.191649


In [135]:
# 2.3 Left join the master panel with the BEA GDP and personal income data
bea = pd.read_csv(
    "/Users/koacow/BOSTON UNIVERSITY Dropbox/Ngoc Duy Khoa Cao/GLOB~S/Data/U.S. County Data/county_controls/data/BEA_COUNTY_1969_2023/bea_county_income_gdp_1980_2022.csv",
    dtype={"FIPS5": str, "year": int},
)
master_panel = master_panel.merge(
    bea,
    on=["FIPS5", "year"],
    how="left",
    suffixes=("", "_bea"),
)
master_panel

,FIPS5,year,median_household_income,total_pop,white_pct,black_pct,asian_pct,other_pct,bachelors_pct,rgdp,personal_income
0,01001,1980,NaN,32259.0,0.769212,0.223969,0.002542,0.004278,0.065625,NaN,250067.0
1,01001,1981,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,261935.0
2,01001,1982,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,280188.0
3,01001,1983,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,302017.0
4,01001,1984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,337459.0
...,...,...,...,...,...,...,...,...,...,...,...
139487,72153,2018,14954.0,36439.0,0.773594,0.035676,0.000137,0.190592,0.175252,NaN,NaN
139488,72153,2019,14743.0,35428.0,0.751016,0.037993,0.000169,0.210822,0.177938,NaN,NaN
139489,72153,2020,14813,34172.0,0.213976,0.047319,0.000732,0.737973,0.180147,NaN,NaN
139490,72153,2021,16444,34704.0,0.706921,0.036941,0.000000,0.256138,0.191649,NaN,NaN


In [136]:
# 2.4 Left join the master panel with the BLS unemployment data
bls = pd.read_csv(
    "/Users/koacow/BOSTON UNIVERSITY Dropbox/Ngoc Duy Khoa Cao/GLOB~S/Data/U.S. County Data/county_controls/data/BLSLAUCN_1990_2022/bls_county_unemp_1990_2022.csv",
    dtype={"FIPS5": str, "year": int},
)
master_panel = master_panel.merge(
    bls,
    on=["FIPS5", "year"],
    how="left",
    suffixes=("", "_bls"),
)
master_panel


,FIPS5,year,median_household_income,total_pop,white_pct,black_pct,asian_pct,other_pct,bachelors_pct,rgdp,personal_income,unemployment_rate
0,01001,1980,NaN,32259.0,0.769212,0.223969,0.002542,0.004278,0.065625,NaN,250067.0,NaN
1,01001,1981,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,261935.0,NaN
2,01001,1982,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,280188.0,NaN
3,01001,1983,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,302017.0,NaN
4,01001,1984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,337459.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
139487,72153,2018,14954.0,36439.0,0.773594,0.035676,0.000137,0.190592,0.175252,NaN,NaN,13.1
139488,72153,2019,14743.0,35428.0,0.751016,0.037993,0.000169,0.210822,0.177938,NaN,NaN,13.2
139489,72153,2020,14813,34172.0,0.213976,0.047319,0.000732,0.737973,0.180147,NaN,NaN,NaN
139490,72153,2021,16444,34704.0,0.706921,0.036941,0.000000,0.256138,0.191649,NaN,NaN,10.5


In [137]:
# 2.5 Left join the master panel with the USDA Rural-Urban Continuum Codes data
rucc = pd.read_csv(
    "/Users/koacow/BOSTON UNIVERSITY Dropbox/Ngoc Duy Khoa Cao/GLOB~S/Data/U.S. County Data/county_controls/data/USDA_ERS_RUCC_2013/usda_rural_urban_codes.csv",
    dtype={"FIPS": str},
)
rucc = rucc.rename(columns={"FIPS": "FIPS5"})
rucc = rucc[["FIPS5", "RUCC_2013"]]
master_panel = master_panel.merge(
    rucc,
    on="FIPS5",
    how="left",
)
master_panel

,FIPS5,year,median_household_income,total_pop,white_pct,black_pct,asian_pct,other_pct,bachelors_pct,rgdp,personal_income,unemployment_rate,RUCC_2013
0,01001,1980,NaN,32259.0,0.769212,0.223969,0.002542,0.004278,0.065625,NaN,250067.0,NaN,2.0
1,01001,1981,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,261935.0,NaN,2.0
2,01001,1982,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,280188.0,NaN,2.0
3,01001,1983,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,302017.0,NaN,2.0
4,01001,1984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,337459.0,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
139487,72153,2018,14954.0,36439.0,0.773594,0.035676,0.000137,0.190592,0.175252,NaN,NaN,13.1,2.0
139488,72153,2019,14743.0,35428.0,0.751016,0.037993,0.000169,0.210822,0.177938,NaN,NaN,13.2,2.0
139489,72153,2020,14813,34172.0,0.213976,0.047319,0.000732,0.737973,0.180147,NaN,NaN,NaN,2.0
139490,72153,2021,16444,34704.0,0.706921,0.036941,0.000000,0.256138,0.191649,NaN,NaN,10.5,2.0


In [138]:
missing_summary_before_interpolation = pd.DataFrame({
    "Missing (%)": master_panel.isna().mean() * 100,
    "Non-Missing (%)": master_panel.notna().mean() * 100,
    "Null Count": master_panel.isnull().sum(),
    "Non-Null Count": master_panel.notnull().sum()
})
missing_summary_before_interpolation.to_markdown(
    "/Users/koacow/repos/glob-s-RA/patents/county_panel/missing_summary_before_interpolation.md"
)
missing_summary_before_interpolation

,Missing (%),Non-Missing (%),Null Count,Non-Null Count
FIPS5,0.000000,100.000000,0,139492
year,0.000000,100.000000,0,139492
median_household_income,69.986809,30.013191,97626,41866
total_pop,63.240903,36.759097,88216,51276
white_pct,63.240903,36.759097,88216,51276
black_pct,63.240903,36.759097,88216,51276
asian_pct,63.240903,36.759097,88216,51276
other_pct,63.240903,36.759097,88216,51276
bachelors_pct,67.866258,32.133742,94668,44824
rgdp,51.291113,48.708887,71547,67945


In [139]:
# 3. Fill in missing values
master_panel = master_panel.sort_values(by=["FIPS5", "year"])

# 3.1 Fill missing values for ACS data. Leave large gaps as NaN
master_panel["total_pop"] = master_panel.groupby("FIPS5")["total_pop"].transform(
    lambda x: x.interpolate(method="linear", limit_direction="both")
)
master_panel["median_household_income"] = master_panel.groupby("FIPS5")["median_household_income"].transform(
    lambda x: x.interpolate(method="linear", limit_direction="both")
) 
master_panel["white_pct"] = master_panel.groupby("FIPS5")["white_pct"].transform(
    lambda x: x.interpolate(method="linear", limit_direction="both")
)
master_panel["black_pct"] = master_panel.groupby("FIPS5")["black_pct"].transform(
    lambda x: x.interpolate(method="linear", limit_direction="both")
)
master_panel["asian_pct"] = master_panel.groupby("FIPS5")["asian_pct"].transform(
    lambda x: x.interpolate(method="linear", limit_direction="both")
)
master_panel["other_pct"] = master_panel.groupby("FIPS5")["other_pct"].transform(
    lambda x: x.interpolate(method="linear", limit_direction="both")
)
master_panel["bachelors_pct"] = master_panel.groupby("FIPS5")["bachelors_pct"].transform(
    lambda x: x.interpolate(method="linear", limit_direction="both")
)

# 3.2 Fill missing values for BEA GDP and personal income data. Leave large gaps of more than 2 years as NaN
master_panel["personal_income"] = master_panel.groupby("FIPS5")["personal_income"].transform(
    lambda x: x.interpolate(method="linear", limit_direction="both", limit=2)
)
master_panel["rgdp"] = master_panel.groupby("FIPS5")["rgdp"].transform(
    lambda x: x.interpolate(method="linear", limit_direction="both", limit=2)
)

# 3.3 Fill missing values for BLS unemployment data. Leave gaps of more than 2 years as NaN.
master_panel["unemployment_rate"] = master_panel.groupby("FIPS5")["unemployment_rate"].transform(
    lambda x: x.interpolate(method="linear", limit_direction="both", limit=2)
)
master_panel

/var/folders/zx/x6hb1_p95p33hh3235qrnlqm0000gn/T/ipykernel_38364/2282229780.py:9: FutureWarning: Series.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  lambda x: x.interpolate(method="linear", limit_direction="both")


,FIPS5,year,median_household_income,total_pop,white_pct,black_pct,asian_pct,other_pct,bachelors_pct,rgdp,personal_income,unemployment_rate,RUCC_2013
0,01001,1980,NaN,32259.0,0.769212,0.223969,0.002542,0.004278,0.065625,NaN,250067.0,NaN,2.0
1,01001,1981,NaN,32455.3,0.771608,0.221573,0.002638,0.004180,0.067905,NaN,261935.0,NaN,2.0
2,01001,1982,NaN,32651.6,0.774004,0.219178,0.002735,0.004083,0.070185,NaN,280188.0,NaN,2.0
3,01001,1983,NaN,32847.9,0.776400,0.216783,0.002831,0.003985,0.072464,NaN,302017.0,NaN,2.0
4,01001,1984,NaN,33044.2,0.778797,0.214388,0.002928,0.003888,0.074744,NaN,337459.0,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
139487,72153,2018,14954.0,36439.0,0.773594,0.035676,0.000137,0.190592,0.175252,NaN,NaN,13.10,2.0
139488,72153,2019,14743.0,35428.0,0.751016,0.037993,0.000169,0.210822,0.177938,NaN,NaN,13.20,2.0
139489,72153,2020,14813,34172.0,0.213976,0.047319,0.000732,0.737973,0.180147,NaN,NaN,11.85,2.0
139490,72153,2021,16444,34704.0,0.706921,0.036941,0.000000,0.256138,0.191649,NaN,NaN,10.50,2.0


In [140]:
missing_summary_after_interpolation = pd.DataFrame({
    "Missing (%)": master_panel.isna().mean() * 100,
    "Non-Missing (%)": master_panel.notna().mean() * 100,
    "Null Count": master_panel.isnull().sum(),
    "Non-Null Count": master_panel.notnull().sum()
})
missing_summary_after_interpolation.to_markdown(
    "/Users/koacow/repos/glob-s-RA/patents/county_panel/missing_summary_after_interpolation.md"
)
missing_summary_after_interpolation

,Missing (%),Non-Missing (%),Null Count,Non-Null Count
FIPS5,0.000000,100.000000,0,139492
year,0.000000,100.000000,0,139492
median_household_income,69.986809,30.013191,97626,41866
total_pop,0.000000,100.000000,0,139492
white_pct,0.000000,100.000000,0,139492
black_pct,0.000000,100.000000,0,139492
asian_pct,0.000000,100.000000,0,139492
other_pct,0.000000,100.000000,0,139492
bachelors_pct,0.092478,99.907522,129,139363
rgdp,46.849282,53.150718,65351,74141


In [141]:
print("Missing summary after interpolation (2012 and after): ")
missing_summary_after_interpolation_2012_and_after = pd.DataFrame({
    "Missing (%)": master_panel[master_panel["year"] >= 2012].isna().mean() * 100,
    "Non-Missing (%)": master_panel[master_panel["year"] >= 2012].notna().mean() * 100,
    "Null Count": master_panel[master_panel["year"] >= 2012].isnull().sum(),
    "Non-Null Count": master_panel[master_panel["year"] >= 2012].notnull().sum()
})
missing_summary_after_interpolation_2012_and_after.to_markdown(
    "/Users/koacow/repos/glob-s-RA/patents/county_panel/missing_summary_after_interpolation_2012_and_after.md"
)
missing_summary_after_interpolation_2012_and_after

Missing summary after interpolation (2012 and after): 


,Missing (%),Non-Missing (%),Null Count,Non-Null Count
FIPS5,0.000000,100.000000,0,35684
year,0.000000,100.000000,0,35684
median_household_income,0.728618,99.271382,260,35424
total_pop,0.000000,100.000000,0,35684
white_pct,0.000000,100.000000,0,35684
black_pct,0.000000,100.000000,0,35684
asian_pct,0.000000,100.000000,0,35684
other_pct,0.000000,100.000000,0,35684
bachelors_pct,0.092478,99.907522,33,35651
rgdp,4.752830,95.247170,1696,33988


In [147]:
master_panel["FIPS5"].unique().shape[0]

3244

In [150]:
summary_stats = master_panel.drop(columns=['FIPS5', 'year', 'RUCC_2013']).describe()
summary_stats.to_markdown(
    "/Users/koacow/repos/glob-s-RA/patents/county_panel/summary_stats.md"
)
summary_stats

,total_pop,white_pct,black_pct,asian_pct,other_pct,bachelors_pct,rgdp,personal_income,unemployment_rate
count,1.394920e+05,139492.000000,139492.000000,139492.000000,139492.000000,139363.000000,7.414100e+04,1.327330e+05,112677.000000
mean,8.952820e+04,0.842537,0.087959,0.009335,0.060169,0.111278,5.574190e+06,3.098990e+06,6.177718
std,2.903377e+05,0.168518,0.142941,0.025791,0.101001,0.058555,2.421519e+07,1.306131e+07,3.197466
min,4.800000e+01,0.020674,0.000000,0.000000,0.000000,0.000000,5.420000e+03,1.610000e+03,0.400000
25%,1.088400e+04,0.768208,0.003489,0.001715,0.010670,0.070844,3.451370e+05,2.078460e+05,4.000000
50%,2.449440e+04,0.907982,0.019337,0.003496,0.026442,0.097360,8.954660e+05,5.327560e+05,5.400000
75%,6.042620e+04,0.966772,0.102698,0.007821,0.065637,0.135398,2.632145e+06,1.580269e+06,7.500000
max,1.010572e+07,1.000000,0.874548,0.798611,0.974665,2.085366,7.937170e+08,7.229358e+08,40.600000


In [142]:
master_panel.dropna()['year'].value_counts().sort_index()

year
2010    3079
2011    3079
2012    3079
2013    3079
2014    3079
2015    3078
2016    3079
2017    3079
2018    3078
2019    3079
2020    3078
2021    3078
2022    3078
Name: count, dtype: int64

In [143]:
# 4. Data Quality Check

# 4.1 Assert that total U.S. population in each ACS/SF1 year matches the expected value
true_data_years = ACS_SF1["year"].unique()
for year in true_data_years:
    panel_us_pop = master_panel[master_panel["year"] == year]["total_pop"].sum()
    true_us_pop = ACS_SF1[ACS_SF1["year"] == year]["total_pop"].sum()
    assert abs(panel_us_pop - true_us_pop) / true_us_pop < 0.05, f"Total US population in {year} deviates more than 5% from expected value."

# 4.2 Check the total U.S. population for all years
total_us_pop = master_panel.groupby("year")["total_pop"].sum()
print("Total U.S. population for each year:")
print(total_us_pop)


# 4.3 Assert that population for specific counties in 2010 matches the expected values (from the US Census Bureau Website)
counties_to_check = ["06037", "17031", "36061"] # Los Angeles County, Cook County, New York County
expected_2020_pops = [
    10014009,  # Los Angeles County
    5275541, # Cook County
    1694251, # New York County
]
for county, expected_pop in zip(counties_to_check, expected_2020_pops):
    panel_pop = master_panel[(master_panel["FIPS5"] == county) & (master_panel["year"] == 2020)]["total_pop"].values[0]
    assert abs(panel_pop - expected_pop) / expected_pop < 0.05, f"Population for {county} in 2020 deviates more than 5% from expected value."

# 4.4 Check unemployment rate, rgdp, personal income, and median household income for each state in all years
state_panel = master_panel.copy().dropna()
state_panel["STATE_FIPS"] = state_panel["FIPS5"].str[:2]
state_panel["unemployed_pop"] = state_panel["total_pop"] * state_panel["unemployment_rate"] / 100
state_panel = state_panel.groupby(["STATE_FIPS", "year"]).agg(
    total_pop=("total_pop", "sum"),
    unemployed_pop=("unemployed_pop", "sum"),
    rgdp=("rgdp", "mean"),
    personal_income=("personal_income", "mean"),
).reset_index()
state_panel["unemployment_rate"] = (
    state_panel["unemployed_pop"] / state_panel["total_pop"] * 100
)
california = state_panel[state_panel["STATE_FIPS"] == "06"]
fred_california_data = pd.read_csv(
    "/Users/koacow/Downloads/CAUR.csv")
fred_california_data["observation_date"] = pd.to_datetime(fred_california_data["observation_date"])
fred_california_data["year"] = fred_california_data["observation_date"].dt.year
fred_california_data = fred_california_data.rename(columns={"CAUR": "unemployment_rate"})
fred_california_data = fred_california_data[["year", "unemployment_rate"]]
california = california.merge(
    fred_california_data,
    on="year",
    how="left",
    suffixes=("", "_fred"),
)
california["unemployment_rate_diff"] = abs(california["unemployment_rate"] - california["unemployment_rate_fred"]) / california["unemployment_rate_fred"]
assert california["unemployment_rate_diff"].max() < 0.05, "Unemployment rate for California deviates more than 5% from FRED data."

print("All data quality checks passed successfully!")

Total U.S. population for each year:
year
1980    236302307.0
1981    238514237.6
1982    240726168.2
1983    242938098.8
1984    245150029.4
1985    247361960.0
1986    249573890.6
1987    251785821.2
1988    253997751.8
1989    256209682.4
1990    258421613.0
1991    261661719.2
1992    264901825.4
1993    268141931.6
1994    271382037.8
1995    274622144.0
1996    277862250.2
1997    281102356.4
1998    284342462.6
1999    287582568.8
2000    290822675.0
2001    293550018.7
2002    296277362.4
2003    299004706.1
2004    301732049.8
2005    304459393.5
2006    307186737.2
2007    309914080.9
2008    312641424.6
2009    315368768.3
2010    318096112.0
2011    315972193.4
2012    318481050.8
2013    320845279.2
2014    323378014.6
2015    325730114.0
2016    327720162.4
2017    330106421.8
2018    331923282.2
2019    333649796.6
2020    340365232.0
2021    338667398.0
2022    339994109.0
Name: total_pop, dtype: float64
All data quality checks passed successfully!


In [144]:
# 4.5 Spot check the data for LA County in 2020
print("Spot check for Los Angeles County in 2020:")
la_county_data = master_panel[(master_panel["FIPS5"] == "06037") & (master_panel["year"] == 2020)]
la_county_data

Spot check for Los Angeles County in 2020:


,FIPS5,year,median_household_income,total_pop,white_pct,black_pct,asian_pct,other_pct,bachelors_pct,rgdp,personal_income,unemployment_rate,RUCC_2013
9199,06037,2020,71358,10014009.0,0.325487,0.079325,0.149789,0.445399,0.231444,729068982.0,678548600.0,12.3,1.0


In [145]:
master_panel.to_parquet(
    "/Users/koacow/BOSTON UNIVERSITY Dropbox/Ngoc Duy Khoa Cao/GLOB~S/Data/U.S. County Data/county_controls/county_controls_1980_2022.parquet",
    index=False,
)